# KPI Inference
This notebook takes in the relevant paragraphs to KPIs found in the relevance infer stage, the fine tuned KPI EXTRACTION model from the training stage, and performs inference to return specific answers to the KPIs.

In [1]:
from config_qa_farm_train import QAFileConfig, QAInferConfig
import pprint
import pathlib
import os
from src.data.s3_communication import S3Communication, S3FileType
from src.models.text_kpi_infer import TextKPIInfer
from dotenv import load_dotenv
import zipfile
import config

07/02/2022 00:30:33 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
# Load credentials
dotenv_dir = os.environ.get(
    "CREDENTIAL_DOTENV_DIR", os.environ.get("PWD", "/opt/app-root/src")
)
dotenv_path = pathlib.Path(dotenv_dir) / "credentials.env"
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

In [3]:
# init s3 connector
s3c = S3Communication(
    s3_endpoint_url=os.getenv("S3_ENDPOINT"),
    aws_access_key_id=os.getenv("S3_LANDING_ACCESS_KEY"),
    aws_secret_access_key=os.getenv("S3_LANDING_SECRET_KEY"),
    s3_bucket=os.getenv("S3_BUCKET"),
)

In [4]:
#Settings data files and checkpoints parameters
file_config = QAFileConfig("infer_demo")
infer_config = QAInferConfig("infer_demo")

In [5]:
# When running in Automation using Elyra and Kubeflow Pipelines,
# set AUTOMATION = 1 as an environment variable
if os.getenv("AUTOMATION"):

    # inference results dir
    if not os.path.exists(infer_config.relevance_dir['Text']):
        pathlib.Path(infer_config.relevance_dir['Text']).mkdir(parents=True, exist_ok=True)

    # kpi inference results dir
    if not os.path.exists(infer_config.result_dir['Text']):
        pathlib.Path(infer_config.result_dir['Text']).mkdir(parents=True, exist_ok=True)

    # load dir
    if not os.path.exists(infer_config.load_dir['Text']):
        pathlib.Path(infer_config.load_dir['Text']).mkdir(parents=True, exist_ok=True)

    # download relevance predictions from s3
    s3c.download_files_in_prefix_to_dir(
        config.BASE_INFER_RELEVANCE_S3_PREFIX,
        infer_config.relevance_dir['Text'],
    )

In [6]:
model_root = pathlib.Path(file_config.saved_models_dir).parent
model_rel_zip = pathlib.Path(model_root, 'KPI_EXTRACTION.zip')
s3c.download_file_from_s3(model_rel_zip, config.CHECKPOINT_S3_PREFIX, "KPI_EXTRACTION.zip")
with zipfile.ZipFile(pathlib.Path(model_root, 'KPI_EXTRACTION.zip'), 'r') as z:
    z.extractall(model_root)

## Inference

We can use the saved model and test it on some real examples.<br><br>
First let's load the model:

In [7]:
file_config.saved_models_dir

'/opt/app-root/src/aicoe-osc-demo/models/KPI_EXTRACTION'

In [8]:
tki = TextKPIInfer(infer_config)

07/02/2022 00:31:27 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}


07/02/2022 00:31:31 - WARNING - farm.infer -   QAInferencer always has task_type='question_answering' even if another value is provided to Inferencer.load() or QAInferencer()


Now, let's make prediction on a pair of paragraph and question.

In [9]:
context = """the paris agreement on climate change drafted in 2015 aims to reduce worldwide emissions of greenhouse
gases to a level intended to limit a rise in global temperatures to below 2 degrees or, better still,
to below 1.5 degrees. verbund’s target of reducing greenhouse gas emissions by 90% measured beginning from
the basis year 2011 5 million tonnes co2e until 2021 includes scope 1, scope 2 market- based and parts of scope 3 emissions
for energy and air travel. the science based targets initiative validated this goal as science-based in october 2016,
i.e. it meets global standards. according to current planning, the target can be achieved.
however, if the grid operator requires higher generation volumes
"""
question = "What is the target year for climate commitment?"


In [10]:
QA_input = [
    {
        "qas": [question],
        "context":  context
    }
]

result = tki.infer_on_dict(QA_input)[0]
pprint.pprint(result)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 17.59 Batches/s]

{'predictions': [{'answers': [{'answer': '2021',
                               'context': 'the basis year 2011 5 million '
                                          'tonnes co2e until 2021 includes '
                                          'scope 1, scope 2 market- based and '
                                          'par',
                               'document_id': '0-0',
                               'offset_answer_end': 364,
                               'offset_answer_start': 360,
                               'offset_context_end': 412,
                               'offset_context_start': 312,
                               'probability': None,
                               'score': 7.129114151000977},
                              {'answer': 'no_answer',
                               'context': '',
                               'document_id': '0-0',
                               'offset_answer_end': 0,
                               'offset_answer_start': 0,
      

What does the prediction result show? 

In [11]:
# This is the best answer. Generally it can be span-based or it can be no-answer, which ever is higher
# Here the top answer is the span '2021'
result['predictions'][0]['answers'][0]

{'score': 7.129114151000977,
 'probability': None,
 'answer': '2021',
 'offset_answer_start': 360,
 'offset_answer_end': 364,
 'context': 'the basis year 2011 5 million tonnes co2e until 2021 includes scope 1, scope 2 market- based and par',
 'offset_context_start': 312,
 'offset_context_end': 412,
 'document_id': '0-0'}

In [12]:
# Non-answerable score: The model is pretty confident that the answer to the question can be in the context.
result['predictions'][0]['answers'][1]

{'score': -20.135552406311035,
 'probability': None,
 'answer': 'no_answer',
 'offset_answer_start': 0,
 'offset_answer_end': 0,
 'context': '',
 'offset_context_start': 0,
 'offset_context_end': 0,
 'document_id': '0-0'}

Now, let's use the model to infer kpi answers from the relevance results 

In [13]:
infer_config.relevance_dir

{'Text': '/opt/app-root/src/aicoe-osc-demo/data/infer_relevance'}

In [14]:
kpi_df = s3c.download_df_from_s3(
    "aicoe-osc-demo/kpi_mapping",
    "kpi_mapping.csv",
    filetype=S3FileType.CSV,
    header=0,
)
kpi_df.head()

,kpi_id,question,sectors,add_year,kpi_category,Unnamed: 5,Unnamed: 6
0,0.0,What is the company name?,"OG, CM, CU",False,TEXT,NaN,NaN
1,1.0,In which year was the annual report or the sus...,"OG, CM, CU",False,TEXT,NaN,NaN
2,2.0,What is the total volume of proven and probabl...,OG,True,"TEXT, TABLE",NaN,NaN
3,2.1,What is the volume of estimated proven hydroca...,OG,True,"TEXT, TABLE",NaN,NaN
4,2.2,What is the volume of estimated probable hydro...,OG,True,"TEXT, TABLE",NaN,NaN


In [15]:
tki.infer_on_relevance_results(infer_config.relevance_dir['Text'], kpi_df)

07/02/2022 00:31:32 - INFO - src.models.text_kpi_infer -   #################### Starting KPI Inference for the following relevance CSV files found in /opt/app-root/src/aicoe-osc-demo/data/infer_kpi:
['sustainability-report-2019_predictions_relevant.csv'] 


07/02/2022 00:31:32 - INFO - src.models.text_kpi_infer -   #################### 1/1


07/02/2022 00:31:32 - INFO - src.models.text_kpi_infer -   Starting KPI Extraction for sustainability-report-2019


Inferencing Samples:   0%|          | 0/35 [00:00<?, ? Batches/s]

Inferencing Samples:   3%|▎         | 1/35 [00:00<00:08,  3.97 Batches/s]

...

Inferencing Samples: 100%|██████████| 35/35 [00:08<00:00,  3.92 Batches/s]

Inferencing Samples: 100%|██████████| 35/35 [00:08<00:00,  4.12 Batches/s]


07/02/2022 00:31:42 - ERROR - farm.modeling.predictions -   Both start and end offsets should be 0: 
86, 86 with a no_answer. 


07/02/2022 00:31:43 - INFO - src.models.text_kpi_infer -   Save the result of KPI extraction to /opt/app-root/src/aicoe-osc-demo/data/infer_kpi/sustainability-report-2019_predictions_kpi.csv


,pdf_name,kpi,kpi_id,answer,page,paragraph,source,score,no_ans_score,no_answer_score_plus_boost
0,sustainability-report-2019,In which year was the annual report or the sus...,NaN,2019,3.0,This report focuses on the sustainability topi...,Text,12.819071,-11.384018,-26.384018
1,sustainability-report-2019,In which year was the annual report or the sus...,NaN,2018,7.0,According to IPCC’s 1.5 C report from 2018 and...,Text,12.508750,-6.967497,-21.967497
2,sustainability-report-2019,In which year was the annual report or the sus...,NaN,2019,26.0,Equinor Sustainability report 2019 High value ...,Text,12.427496,-9.680325,-24.680325
3,sustainability-report-2019,In which year was the annual report or the sus...,NaN,2019,8.0,Equinor Sustainability report 2019Low carbon —...,Text,12.356202,-8.748007,-23.748007
4,sustainability-report-2019,What is the annual total production from coal?,NaN,no_answer,NaN,NaN,Text,2.840454,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
91,sustainability-report-2019,What is the volume of estimated probable hydro...,NaN,18kg CO₂/boe.,10.0,Equinor aims to reduce the CO₂ intensity of it...,Text,-6.069309,17.698502,2.698502
92,sustainability-report-2019,What is the volume of estimated proven hydroca...,NaN,2.7 billion barrels of oil equivalents,25.0,Norway This year Equinor and the Johan Sverdru...,Text,5.207407,17.570000,2.570000
93,sustainability-report-2019,What is the volume of estimated proven hydroca...,NaN,18kg CO₂/boe.,10.0,Equinor aims to reduce the CO₂ intensity of it...,Text,-6.862871,17.566399,2.566399
94,sustainability-report-2019,What is the volume of estimated proven hydroca...,NaN,8 744 m3,18.0,The number of oil spills per year decreased co...,Text,-7.836313,17.525288,2.525288


In [16]:
if os.getenv("AUTOMATION"):
    # upload the predicted files to s3
    s3c.upload_files_in_dir_to_prefix(
        infer_config.result_dir['Text'],
        config.BASE_INFER_KPI_S3_PREFIX
    )

# Conclusion
This notebook ran the _KPI_ inference on a sample dataset and stored the output in a csv format.